In [1]:
from google.colab import drive
import json
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [15]:
import torch
import torch.nn as nn
from torchvision.models.vgg import vgg19
from torch.optim.adam import Adam
from torchvision.utils import save_image
import torch.nn.functional as F
from torch.utils.data import DataLoader
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

import time
import os
from random import seed
import math
from util import *

seed(11785)

In [60]:
# define G, D, RRDB, perceptual loss
class ResidualBlock(nn.Module):
    def __init__(self, nf, gc=32, res_scale=0.2):
        super(ResidualBlock, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(nf + 0 * gc, gc, 3, padding=1, bias=True), nn.LeakyReLU())
        self.layer2 = nn.Sequential(nn.Conv2d(nf + 1 * gc, gc, 3, padding=1, bias=True), nn.LeakyReLU())
        self.layer3 = nn.Sequential(nn.Conv2d(nf + 2 * gc, gc, 3, padding=1, bias=True), nn.LeakyReLU())
        self.layer4 = nn.Sequential(nn.Conv2d(nf + 3 * gc, gc, 3, padding=1, bias=True), nn.LeakyReLU())
        self.layer5 = nn.Sequential(nn.Conv2d(nf + 4 * gc, nf, 3, padding=1, bias=True), nn.LeakyReLU())

        self.res_scale = res_scale

    def forward(self, x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(torch.cat((x, layer1), 1))
        layer3 = self.layer3(torch.cat((x, layer1, layer2), 1))
        layer4 = self.layer4(torch.cat((x, layer1, layer2, layer3), 1))
        layer5 = self.layer5(torch.cat((x, layer1, layer2, layer3, layer4), 1))
        return layer5.mul(self.res_scale) + x


class RRDB(nn.Module):
    def __init__(self, nf, gc=32, res_scale=0.2):
        super(RRDB, self).__init__()
        self.layers = nn.Sequential(ResidualBlock(nf, gc),
                                    ResidualBlock(nf, gc),
                                    ResidualBlock(nf, gc, ))
        self.res_scale = res_scale

    def forward(self, x):
        out = self.layers(x)
        return out.mul(self.res_scale) + x


def Upsamlper(nf, scale_factor=2):
    layers = []
    for _ in range(scale_factor//2):
        layers += [
            nn.Conv2d(nf, nf * (2 ** 2), 1),
            nn.PixelShuffle(2),
            nn.ReLU()]
    return nn.Sequential(*layers)

class Discriminator(nn.Module):
    def __init__(self, num_conv_block=4):
        super(Discriminator, self).__init__()

        layers = []
        in_channels = 3
        out_channels = 64
        for _ in range(num_conv_block):
            layers += [nn.ReflectionPad2d(1),
                      nn.Conv2d(in_channels, out_channels, 3),
                      nn.LeakyReLU(),
                      nn.BatchNorm2d(out_channels)]
            in_channels = out_channels

            layers += [nn.ReflectionPad2d(1),
                      nn.Conv2d(in_channels, out_channels, 3, 2),
                      nn.LeakyReLU()]
            out_channels *= 2

        out_channels //= 2
        in_channels = out_channels

        layers += [nn.Conv2d(in_channels, out_channels, 3),
                  nn.LeakyReLU(0.3),
                  nn.Conv2d(out_channels, out_channels, 3)]

        self.layers = nn.Sequential(*layers)
        self.linear = nn.Sequential(
            nn.Linear(8192, 100),
            nn.Linear(100, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

class Generator(nn.Module):
    def __init__(self, in_channels, out_channels, nf=64, gc=32, scale_factor=4, n_basic_block=23):
        super(Generator, self).__init__()

        self.conv1 = nn.Sequential(nn.ReflectionPad2d(1), nn.Conv2d(in_channels, nf, 3), nn.ReLU())

        basic_block_layer = []

        for _ in range(n_basic_block):
            basic_block_layer += [RRDB(nf, gc)]

        self.basic_block = nn.Sequential(*basic_block_layer)

        self.conv2 = nn.Sequential(nn.ReflectionPad2d(1), nn.Conv2d(nf, nf, 3), nn.ReLU())
        self.upsample = Upsamlper(nf, scale_factor=scale_factor)
        self.conv3 = nn.Sequential(nn.ReflectionPad2d(1), 
                                   nn.Conv2d(nf, nf, 3), 
                                   nn.ReLU(),
                                   nn.ReflectionPad2d(1), 
                                   nn.Conv2d(nf, out_channels, 3), 
                                   nn.ReLU())
    def forward(self, x):
        x1 = self.conv1(x)
        x = self.basic_block(x1)
        x = self.conv2(x)
        x = self.upsample(x + x1)
        x = self.conv3(x)
        return x
        
class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()

        vgg = vgg19(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
        for p in loss_network.parameters():
            p.requires_grad = False
        self.network = loss_network
        self.loss = nn.MSELoss()

    def forward(self, high_resolution, fake_high_resolution):
        perception_loss = self.loss(self.network(fake_high_resolution), self.network(high_resolution))
        return perception_loss

In [61]:
# define train() and validate() in the model
class GAN_model:
    def __init__(self, config, train_loader=None, dev_loader=None, test_loader=None):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_epoch = config.num_epoch
        self.epoch = config.load_model_epoch
        self.image_size = config.image_size
        self.train_loader = train_loader
        self.dev_loader = dev_loader
        self.test_loader = test_loader
        self.checkpoint_dir = config.checkpoint_dir
        self.batch_size = config.batch_size
        self.sample_dir = config.sample_dir
        self.scale_factor = config.scale_factor

        self.psnr_lr = config.psnr_lr
        self.lr = config.lr
        self.content_loss_factor = config.content_loss_factor
        self.perceptual_loss_factor = config.perceptual_loss_factor
        self.adversarial_loss_factor = config.adversarial_loss_factor

        self.generator = Generator(3, 3, 64, scale_factor=self.scale_factor).to(self.device)
        self.discriminator = Discriminator().to(self.device)

        if config.load_model_epoch:
          self.load_model()

        psnr_epoch_indices = math.floor(config.psnr_epochs // 4)
        interval_epoch = math.ceil(self.num_epoch // 8)
        temp = [psnr_epoch_indices, psnr_epoch_indices * 2, psnr_epoch_indices * 4, psnr_epoch_indices * 6]
        epoch_indices = [interval_epoch, interval_epoch * 2, interval_epoch * 4, interval_epoch * 6]
        self.optimizer_psnr = Adam(self.generator.parameters(), lr=self.psnr_lr, betas=(config.b1, config.b2))
        self.optimizer_generator = Adam(self.generator.parameters(), lr=self.lr, betas=(config.b1, config.b2),  # 这里为啥lr和betas是列表
                                        weight_decay=config.weight_decay)
        self.optimizer_discriminator = Adam(self.discriminator.parameters(), lr=self.lr, betas=(config.b1, config.b2),
                                            weight_decay=config.weight_decay)

        self.lr_scheduler_psnr = torch.optim.lr_scheduler.MultiStepLR(self.optimizer_psnr, temp, 0.5)
        self.lr_scheduler_generator = torch.optim.lr_scheduler.MultiStepLR(self.optimizer_generator, epoch_indices, 0.5)
        self.lr_scheduler_discriminator = torch.optim.lr_scheduler.MultiStepLR(self.optimizer_discriminator, epoch_indices, 0.5)

        self.adversarial_criterion = nn.BCEWithLogitsLoss().to(self.device)
        self.content_criterion = nn.L1Loss().to(self.device)
        self.perception_criterion = PerceptualLoss().to(self.device)

    def load_model(self):
        %cd ./gdrive/My Drive/11785/HW5/
        print(f"[*] Load model from google drive {self.checkpoint_dir} epoch {self.epoch}")
        if not os.path.exists(self.checkpoint_dir):
            self.makedirs = os.makedirs(self.checkpoint_dir)

        if not os.listdir(self.checkpoint_dir):
            print(f"[!] No checkpoint in {self.checkpoint_dir}")
            %cd /content
            return

        generator = glob(os.path.join(self.checkpoint_dir, f'generator_{self.epoch}.pth'))
        discriminator = glob(os.path.join(self.checkpoint_dir, f'discriminator_{self.epoch}.pth'))

        if not generator:
            print(f"[!] No checkpoint in epoch {self.epoch - 1}")
            %cd /content
            return

        self.generator.load_state_dict(torch.load(generator[0]))
        self.discriminator.load_state_dict(torch.load(discriminator[0]))

        %cd /content

    def pre_train_psnr(self):
        total_step = len(self.train_loader)
        total_G_loss, total_D_loss = 0, 0
        self.generator.train()

        if not os.path.exists(os.path.join(self.sample_dir, str(epoch))):
            os.makedirs(os.path.join(self.sample_dir, str(epoch)))

        for step, image in enumerate(self.train_loader):
            # if step == 5: break
            low_resolution = image['lr'].to(self.device)
            high_resolution = image['hr'].to(self.device)
            # low.shape = (batch_size, n_channel, 32, 32), high.shape = (batch_size, n_channel, 128, 128)

            real_labels = torch.ones((high_resolution.size(0), 1)).to(self.device)
            fake_labels = torch.zeros((high_resolution.size(0), 1)).to(self.device)

            # pretrain generator
            self.optimizer_psnr.zero_grad()
            fake_high_resolution = self.generator(low_resolution)

            generator_loss = nn.MSELoss(fake_high_resolution, high_resolution)

            generator_loss.backward()
            self.optimizer_psnr.step()

            total_G_loss += generator_loss.item()
        self.lr_scheduler_psnr.step()

        # %cd ./gdrive/My Drive/11785/HW5/
        # torch.save(self.generator.state_dict(), os.path.join(self.checkpoint_dir, f"generator_{epoch}.pth")
        # %cd /content
        return total_G_loss/total_step


    def train(self, epoch):
        total_step = len(self.train_loader)
        total_G_loss, total_D_loss = 0, 0
        self.generator.train()
        self.discriminator.train()

        for step, image in enumerate(self.train_loader):
            # if step == 5: break
            low_resolution = image['lr'].to(self.device)
            high_resolution = image['hr'].to(self.device)
            # low.shape = (batch_size, n_channel, 32, 32), high.shape = (batch_size, n_channel, 128, 128)

            real_labels = torch.ones((high_resolution.size(0), 1)).to(self.device)
            fake_labels = torch.zeros((high_resolution.size(0), 1)).to(self.device)

            #generator
            self.optimizer_generator.zero_grad()
            fake_high_resolution = self.generator(low_resolution)

            fake_prob, real_prob = self.discriminator(high_resolution), self.discriminator(fake_high_resolution)

            adversarial_loss_rf = self.adversarial_criterion(real_prob - fake_prob.mean(), fake_labels)
            adversarial_loss_fr = self.adversarial_criterion(fake_prob - real_prob.mean(), real_labels)
            adversarial_loss = (adversarial_loss_fr + adversarial_loss_rf) / 2

            perceptual_loss = self.perception_criterion(high_resolution, fake_high_resolution)
            content_loss = self.content_criterion(fake_high_resolution, high_resolution)

            generator_loss = adversarial_loss * self.adversarial_loss_factor + \
                              perceptual_loss * self.perceptual_loss_factor + \
                              content_loss * self.content_loss_factor

            generator_loss.backward()
            self.optimizer_generator.step()

            #discriminator
            self.optimizer_discriminator.zero_grad()

            fake_prob, real_prob = self.discriminator(high_resolution), self.discriminator(fake_high_resolution.detach())

            adversarial_loss_rf = self.adversarial_criterion(real_prob - fake_prob.mean(), real_labels)
            adversarial_loss_fr = self.adversarial_criterion(fake_prob - real_prob.mean(), fake_labels)
            discriminator_loss = (adversarial_loss_fr + adversarial_loss_rf) / 2

            discriminator_loss.backward()
            self.optimizer_discriminator.step()

            if step == 0:
                low_resolution_upscale = F.interpolate(low_resolution, size=128)  # 注意这里的128！
                result = torch.cat((high_resolution, fake_high_resolution), 2)
                result = torch.cat((result, low_resolution_upscale), 2)
                save_image(result, os.path.join(self.sample_dir, f"SR_{epoch}.png"))
            
            total_G_loss += generator_loss.item()
            total_D_loss += discriminator_loss.item()

        self.lr_scheduler_generator.step()
        self.lr_scheduler_discriminator.step()

        %cd ./gdrive/My Drive/11785/HW5/
        torch.save(self.generator.state_dict(), os.path.join(self.checkpoint_dir, f"generator_{epoch}.pth"))
        torch.save(self.discriminator.state_dict(), os.path.join(self.checkpoint_dir, f"discriminator_{epoch}.pth"))
        %cd /content

        return total_G_loss/total_step, total_D_loss/total_step

    def validate(self):
      with torch.no_grad():
        total_step = len(self.dev_loader)
        self.generator.eval()
        self.discriminator.eval()

        total_G_loss, total_D_loss = 0, 0
        PSNR, SSIM = 0, 0
        for step, image in enumerate(self.dev_loader):
          # if step == 5: break
          low_resolution = image['lr'].to(self.device)
          high_resolution = image['hr'].to(self.device)
          # low.shape = (batch_size, n_channel, 32, 32), high.shape = (batch_size, n_channel, 128, 128)

          real_labels = torch.ones((high_resolution.size(0), 1)).to(self.device)
          fake_labels = torch.zeros((high_resolution.size(0), 1)).to(self.device)

          #generator
          self.optimizer_generator.zero_grad()
          fake_high_resolution = self.generator(low_resolution)

          fake_prob, real_prob = self.discriminator(high_resolution), self.discriminator(fake_high_resolution)

          adversarial_loss_rf = self.adversarial_criterion(real_prob - fake_prob.mean(), fake_labels)
          adversarial_loss_fr = self.adversarial_criterion(fake_prob - real_prob.mean(), real_labels)
          adversarial_loss = (adversarial_loss_fr + adversarial_loss_rf) / 2

          perceptual_loss = self.perception_criterion(high_resolution, fake_high_resolution)
          content_loss = self.content_criterion(fake_high_resolution, high_resolution)

          generator_loss = adversarial_loss * self.adversarial_loss_factor + \
                            perceptual_loss * self.perceptual_loss_factor + \
                            content_loss * self.content_loss_factor
          self.optimizer_generator.step()

          #discriminator
          self.optimizer_discriminator.zero_grad()

          fake_prob, real_prob = self.discriminator(high_resolution), self.discriminator(fake_high_resolution.detach())

          adversarial_loss_rf = self.adversarial_criterion(real_prob - fake_prob.mean(), real_labels)
          adversarial_loss_fr = self.adversarial_criterion(fake_prob - real_prob.mean(), fake_labels)
          discriminator_loss = (adversarial_loss_fr + adversarial_loss_rf) / 2

          total_G_loss += generator_loss.item()
          total_D_loss += discriminator_loss.item()

        # calculate PSNR and SSIM
          high_resolution = high_resolution.permute(0, 2, 3, 1).cpu().detach().numpy()
          fake_high_resolution = fake_high_resolution.permute(0, 2, 3, 1).cpu().detach().numpy()

          PSNR += peak_signal_noise_ratio(high_resolution[0], fake_high_resolution[0])
          SSIM += structural_similarity(high_resolution[0], fake_high_resolution[0], multichannel=True)

        return total_G_loss/total_step, total_D_loss/total_step, PSNR/total_step, SSIM/total_step

In [62]:
# parameter
class args():
  image_size = 128, # the height / width of the hr image to network
  batch_size = 16
  sample_batch_size = 1  # batch size for generating image
  num_epoch = 60  # number of epochs to train for
  psnr_epochs = 10
  load_model_epoch = False  # epochs in current train (load model), false then start from scratch
  checkpoint_dir = 'checkpoints' #path to saved models
  sample_dir = 'samples'  #folder to output images and model checkpoints
  scale_factor = 4 # scale factor for super resolution
  # nf = 32 #number of filter in esrgan
  b1 = 0.9  # coefficients used for computing running averages of gradient and its square
  b2 = 0.999  #coefficients used for computing running averages of gradient and its square
  weight_decay = 1e-2

  psnr_lr = 2e-4
  lr = 1e-4 #learning rate when when training generator oriented
  content_loss_factor = 1e-1  # content loss factor when training generator oriented
  perceptual_loss_factor = 1
  adversarial_loss_factor = 5e-3
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [63]:
#download the dataset and preprocess the data (crop, resize)
download_dataset()

print('[!] Making Patches')
crop_image('train_hr', 'train_lr', image_size, image_size/4)
resize_image('valid_hr', image_size)
resize_image('valid_lr', image_size/4)

In [59]:
# 主流程，跑全部，存模型
%cd /content
# bmake directory not existed
if args.checkpoint_dir is None:
    args.checkpoint_dir = 'checkpoints'
if not os.path.exists(args.checkpoint_dir):
    os.makedirs(args.checkpoint_dir)
if not os.path.exists(args.sample_dir):
    os.makedirs(args.sample_dir)

print(f"ESRGAN start")

train_dataset = Datasets('train')
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True)
dev_dataset = Datasets('valid')
dev_loader = torch.utils.data.DataLoader(dataset=dev_dataset, batch_size=args.sample_batch_size, shuffle=True)


model = GAN_model(args, train_loader, dev_loader)
for epoch in range(0, args.num_epoch):
  start_time = time.time()
  G_loss, D_loss = model.train(epoch)
  end_time = time.time()

  print('-----------Epoch %d Training Time-------------' % epoch)
  print(f"[D loss {D_loss:.4f}] [G loss {G_loss:.4f}]"
        f"[Duration {(end_time-start_time)/60:.4f}]"
        f"")
  print('\n')

  start_time = time.time()
  G_loss, D_loss, PSNR, SSIM = model.validate()
  end_time = time.time()
  print('-----------Epoch %d Evaluation Time-------------' % epoch)
  print(f"[D loss {D_loss:.4f}] [G loss {G_loss:.4f}]"
        f"[PSNR {PSNR:.4f}]"
        f"[SSIM {SSIM:.4f}]"
        f"[Duration {(end_time-start_time)/60:.4f}]"
        f"")
  print('\n')

/content
ESRGAN start
/content/gdrive/My Drive/11785/HW5
/content
-----------Epoch 0 Training Time-------------
[D loss 0.0039] [G loss 0.0331][Duration 0.0575]


-----------Epoch 0 Evaluation Time-------------
[D loss 0.0347] [G loss 0.5680][PSNR 0.5277][SSIM 0.0060][Duration 0.0087]


/content/gdrive/My Drive/11785/HW5
/content
-----------Epoch 1 Training Time-------------
[D loss 0.0011] [G loss 0.0405][Duration 0.0561]


-----------Epoch 1 Evaluation Time-------------
[D loss 0.0107] [G loss 0.8504][PSNR 0.3733][SSIM 0.0036][Duration 0.0086]


/content/gdrive/My Drive/11785/HW5
/content
-----------Epoch 2 Training Time-------------
[D loss 0.0000] [G loss 0.0365][Duration 0.0568]


-----------Epoch 2 Evaluation Time-------------
[D loss 0.0000] [G loss 0.8803][PSNR 0.3701][SSIM 0.0039][Duration 0.0102]


/content/gdrive/My Drive/11785/HW5
/content
-----------Epoch 3 Training Time-------------
[D loss 0.0000] [G loss 0.0375][Duration 0.0571]


-----------Epoch 3 Evaluation Time-----

KeyboardInterrupt: ignored

In [ ]:
torch.cuda.empty_cache()

In [68]:
%cd /content/gdrive/My Drive/11785/HW5

# change to the google drive folder to generate test image
def test(target_folder, load_model_epoch, args, resize=False): # generate image from testset
    if resize:
      resize_image('test_lr/'+target_folder, resize, 'test_resize/'+target_folder)
      test_dataset = Datasets(mode='test_resize/'+target_folder)
    else:
      test_dataset = Datasets(mode='test_lr/'+target_folder)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=args.sample_batch_size)
    if not os.path.exists('test_results/'+target_folder):
      os.makedirs('test_results/'+target_folder)
    
    model = GAN_model(args, test_loader=test_loader)
    # model.to(args.device)
    model.generator.load_state_dict(torch.load(os.path.join(args.checkpoint_dir, f'generator_{load_model_epoch}.pth')))
    with torch.no_grad():
      model.generator.eval()
      for step, image in enumerate(test_loader):
        lr = image['lr'].to(args.device)
        image_name = image['hr']
        outputs = model.generator(lr)
        save_image(outputs, os.path.join('test_results/'+target_folder, image_name[0]))

target_folder = ['large_test', 'small_test', 'comics', 'structures']
load_model_epoch = 23
for t in target_folder:
    test(t, load_model_epoch, args, resize=384)

/content/gdrive/My Drive/11785/HW5
[*] [0/100] Make patch test_lr/large_test/108070.png
[*] [0/19] Make patch test_lr/small_test/barbara.png
[*] [0/109] Make patch test_lr/comics/AisazuNihaIrarenai.png
[*] [0/100] Make patch test_lr/structures/img_002.png


In [70]:
TEST_LABEL_PATH = 'test_labels/small_test/'
TEST_RESULT_PATH = 'test_results/small_test/'
TEST_RESTORE_PATH =  'test_restore_results/small_test/'


scores = compute_scores(TEST_LABEL_PATH, TEST_RESULT_PATH, TEST_RESTORE_PATH)
print('small_test: ', scores)

TEST_LABEL_PATH = 'test_labels/large_test/'
TEST_RESULT_PATH = 'test_results/large_test/'
TEST_RESTORE_PATH =  'test_restore_results/large_test/'

scores = compute_scores(TEST_LABEL_PATH, TEST_RESULT_PATH, TEST_RESTORE_PATH)
print('large_test: ', scores)

TEST_LABEL_PATH = 'test_labels/comics/'
TEST_RESULT_PATH = 'test_results/comics/'
TEST_RESTORE_PATH =  'test_restore_results/comics/'

scores = compute_scores(TEST_LABEL_PATH, TEST_RESULT_PATH, TEST_RESTORE_PATH)
print('comic: ', scores)

TEST_LABEL_PATH = 'test_labels/structures/'
TEST_RESULT_PATH = 'test_results/structures/'
TEST_RESTORE_PATH =  'test_restore_results/structures/'

scores = compute_scores(TEST_LABEL_PATH, TEST_RESULT_PATH, TEST_RESTORE_PATH)
print('structures: ', scores)

In [ ]:


# SRResnet result

# without resize: 20.105424887688333 0.6729383997153978

# 24: 18.83101057509526 0.5856139937502995

# 96: 20.764082463704487 0.6797901165309642

# 128: 20.978491100856992 0.6822266197132818

# 196: 21.213560813348508 0.6941931686460572


# ESRGAN result

# 64:size
# In computer scores
# In get test images
# 21.45309622385869 0.6802526439816884
# small_test:  None
# In computer scores
# In get test images
# 20.747147799217373 0.6334000266590871
# large_test:  None
# In computer scores
# In get test images
# 16.006150111341537 0.5802311635659595
# comic:  None
# In computer scores
# In get test images
# 16.554238189410377 0.48245588636768943
# structures:  None

# 128:size
# In computer scores
# In get test images
# 23.1183977326426 0.75362138135607
# small_test:  None
# In computer scores
# In get test images
# 21.371081353390437 0.678602171482434
# large_test:  None
# In computer scores
# In get test images
# 18.65251946090006 0.6674142273974711
# comic:  None
# In computer scores
# In get test images
# 19.090746158291886 0.595484765826424
# structures:  None

# 256:size
# In computer scores
# In get test images
# 23.487613735752547 0.7684400402702198
# small_test:  None
# In computer scores
# In get test images
# 21.299195917203264 0.6826661956559363
# large_test:  None
# In computer scores
# In get test images
# 20.69369763089257 0.7666858174277854
# comic:  None
# In computer scores
# In get test images
# 20.81196694237977 0.7011574299367789
# structures:  None

# 384:size
# In computer scores
# In get test images
# 23.53565344322597 0.7697716136107305
# small_test:  None
# In computer scores
# In get test images
# 21.228789973878655 0.6815376732585176
# large_test:  None
# In computer scores
# In get test images
# 21.038829295921786 0.787277712110814
# comic:  None
# In computer scores
# In get test images
# 20.976102972057376 0.704620308515809
# structures:  None

